<a href="https://colab.research.google.com/github/fatday/My-Grad-Math-Works/blob/main/CME/A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np

# Q1

For homogeneous solution:

$$
-u''+u=0 \ \ \ \ \implies \ \ \ \ \lambda_1=1 \ \ \ \lambda_2=-1 \ \ \ \ \implies \ \ \ u_h(x)=C_1e^x+C_2e^{-x}
$$

Now we solve for particular solution, let $u_p(x)=Ax+B$, then

$$
-u''+u=2x \ \ \ \ \implies \ \ \ \ Ax+B=2x \ \ \ \ \implies \ \ \ \ u_p(x)=2x
$$

Then we have the solution

$$
u(x)=u_h(x)+u_p(x)=C_1e^x+C_2e^{-x}+2x
$$

Since $u(0)=u(1)=0$, then solving $C_1,C_2$ we get

$$
u(x)=\frac{2e}{1-e^2}e^x-\frac{2e}{1-e^2}e^{-x}+2x
$$

In [3]:
def exact(h, X_range):
  x = np.array([i * h for i in range(1, int((X_range[1] - X_range[0])/h))])
  coef = (2 * np.e) / (1 - np.e**2)
  return coef * np.e**x - coef * np.e**(-x) + 2*x

In [4]:
def f(x):
  return 2*x

In [5]:
def solve_solution(h,X_range):
  x_vec = np.array([i * h for i in range(1, int((X_range[1] - X_range[0])/h))])
  n = len(x_vec)
  a,b = (2/h) + (2*h/3), (h/6 - 1/h)
  A = ((np.diag(a * np.ones(n), 0)  + np.diag(b * np.ones(n-1), 1) + np.diag(b * np.ones(n-1), -1)))
  fx =  f(x_vec) * h  # (f, Chi) = int f_i * Chi_i=int f_i on [0,1] = f_i
  u = np.linalg.solve(A, fx)
  #print(A)
  return u

In [6]:
X_range = [0, 1]
h1, h2 = 1/10, 1/20
U1 = solve_solution(h1, X_range)
U2 = solve_solution(h2, X_range)

In [7]:
u1 = exact(h1, X_range)
u2 = exact(h2, X_range)

## Max Error at mesh-points for h= 1/10, 1/20

For $h=1/10$: max error is $8.8514\cdot 10^{-5}$

In [8]:
max(abs(U1-u1))

8.85143578538139e-05

For $h=1/20$: max error is  $2.2108\cdot 10^{-5}$

In [9]:
max(abs(U2-u2))

2.2107691947451102e-05

# Q2

In [10]:
def f(x1, x2):
  return np.sin(np.pi * x1) * (np.sin(np.pi * x2) + np.sin(2 * np.pi * x2))


def u(x1, x2):
  coef1, coef2 = 1 / (2 * (np.pi)**2), 1 / (5 * (np.pi)**2)
  base1, base2 = np.sin(np.pi * x1) * np.sin(np.pi * x2), np.sin(np.pi * x1) * np.sin(2 * np.pi * x2)
  return coef1 * base1 + coef2 * base2

In [11]:
def load_vec_lattices(i, j, h, n):
  p0 = np.array([i * h, j * h])

  # ver
  p1 = np.array([(i+1) * h, j * h])
  p2 = np.array([(i - 1) * h, j * h])

  # horz
  p3 = np.array([i * h, (j+1) * h])
  p4 = np.array([i * h, (j-1) * h])

  # diag
  p5 = np.array([(i-1) * h, (j+1) * h])
  p6 = np.array([(i+1) * h, (j-1) * h])

  # tri_mid
  t1 = np.mean([p0, p2, p4], axis=0)
  t2 = np.mean([p0, p2, p5], axis=0)
  t3 = np.mean([p0, p3, p5], axis=0)
  t4 = np.mean([p0, p4, p6], axis=0)
  t5 = np.mean([p0, p1, p6], axis=0)
  t6 = np.mean([p0, p1, p3], axis=0)

  tris =  np.array([t1, t2, t3, t4, t5, t6])
  f_val = sum(f(tris[:, 0], tris[:, 1]))
  return f_val / 3

In [12]:

def Finite_Difference_Approx(h, space):
  Length = space[1] - space[0]
  M = int(Length / h)
  n = M - 1

  # Construct B and A
  B = (np.diag(4.0 * np.ones(n), 0)  + np.diag(-1.0 * np.ones(n-1), 1) + np.diag(-1.0 * np.ones(n-1), -1))
  I = np.eye(n)
  C = np.diag(-1.0 * np.ones(n-1), 1) + np.diag(-1.0 * np.ones(n-1), -1)
  A = (np.kron(B, I) + np.kron(C, I))
  #print(A)
  f_vec = np.zeros(n*n)
  exact_u = np.zeros(n*n)
  for i in range(n):
    for j in range(n):
      x1, x2 = (i + 1) * h, (j + 1) * h # get x1, x2 for h, 2h,...., (M-1)h
      f_vec[j * n + i] = f(x1,x2) * h**2 #load_vec_lattices(i, j, h, n) * h**2 / 2
      exact_u[j * n + i] = u(x1, x2)
  #print(f_vec)
  #print(exact_u)
  estimated_u = np.linalg.solve(A, f_vec)
  estimated_u = estimated_u.reshape((n, n))
  exact_u = exact_u.reshape((n, n))
  return estimated_u, exact_u, A


In [31]:
h1 = 1/10
h2 = 1/20
space = [0, 1]

# h = 1/10
Uj1, u1, A1 = Finite_Difference_Approx(h1, space)

# h = 1/20
Uj2, u2, A2 = Finite_Difference_Approx(h2, space)

In [32]:
Uj1.shape

(9, 9)

In [25]:
np.sqrt(h1**2 /6 * np.sum((Uj1- u1)**2))

0.0014668925462081417

In [21]:
np.sum((Uj1- u1)**2)

0.0012910642452726028

In [44]:
def transofrm(error_matrix):
  fillin = np.zeros((error_matrix.shape[0] + 2, error_matrix.shape[1] + 2))
  fillin[1:-1,1:-1] = error_matrix
  return fillin


In [67]:
def L2(Uj, u, h):
  area = h**2 / 2
  transoformedM = transofrm(Uj)
  transoformedu = transofrm(u)
  n, m = u.shape
  L = 0
  for i in range(1, n - 1):
    for j in range(1, m - 1):
      L += area * method(i, j, Uj, u)
  return np.sqrt(L)

In [71]:
L2(Uj1,u1, h1)

0.002821809772645859

In [72]:
L2(Uj2,u2, h2)

0.0027443005813007353

In [66]:
np.sqrt(h1**2 * e.T @ A1 @ e / 3)

0.0018108882368262844

In [55]:
L2(Uj2, u2, A2)

array([[0.22226135, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.22226135, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.22226135, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.22226135, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.22226135,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.22226135]])

# L2 Norm of the error for $h=1/10$: 0.03593

In [29]:
# calculate L-2 Norm
def L2(Uj, uj, A):
  e = Uj - uj
  return  np.sqrt(e.T @ A @ e / 3)

In [70]:
def method(i, j, transoformedM, transformu):
  p0 = transoformedM[i, j]

  # ver
  p1 = transoformedM[(i+1), j]
  p2 = transoformedM[(i - 1), j]

  # horz
  p3 = transoformedM[i, (j+1)]
  p4 = transoformedM[i, (j-1)]

  # diag
  p5 = transoformedM[(i-1), (j+1)]
  p6 = transoformedM[(i+1), (j-1)]

  # tri_mid
  t1 = np.mean([p0, p2, p4], axis=0)
  t2 = np.mean([p0, p2, p5], axis=0)
  t3 = np.mean([p0, p3, p5], axis=0)
  t4 = np.mean([p0, p4, p6], axis=0)
  t5 = np.mean([p0, p1, p6], axis=0)
  t6 = np.mean([p0, p1, p3], axis=0)

  tris =  np.array([t1, t2, t3, t4, t5, t6])
  f_val = np.sum(tris) / 6 - transformu[i, j]
  return f_val**2

# L2 Norm of the error for $h=1/20$: 0.07495

In [299]:
l2_norm2 = L2(Uj2, u2, A2)
l2_norm2

0.05095943382468167